## What is MediaPipe ?
![image](https://google.github.io/mediapipe/images/mediapipe_small.png)

MediaPipe offers ready-to-use yet customizable Python solutions as a prebuilt Python package.

Refer - [here](https://google.github.io/mediapipe/)

## Step 1: Install Libraries and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle

## Step 2: Define Utils

In [2]:
mp_drawing = mp.solutions.drawing_utils # For Drawing the cordinates
mp_hands = mp.solutions.hands # Solution specific for hand coordinate

## Step 3: Make some Detection

In [6]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



#         image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

## Step 4: Capture Landmarks & Export to CSV

In [3]:
import csv
import os
import numpy as np

In [7]:
num_coords = len(hand_landmarks.landmark)
num_coords

21

In [21]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [22]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'x2',
 'y2',
 'z2',
 'x3',
 'y3',
 'z3',
 'x4',
 'y4',
 'z4',
 'x5',
 'y5',
 'z5',
 'x6',
 'y6',
 'z6',
 'x7',
 'y7',
 'z7',
 'x8',
 'y8',
 'z8',
 'x9',
 'y9',
 'z9',
 'x10',
 'y10',
 'z10',
 'x11',
 'y11',
 'z11',
 'x12',
 'y12',
 'z12',
 'x13',
 'y13',
 'z13',
 'x14',
 'y14',
 'z14',
 'x15',
 'y15',
 'z15',
 'x16',
 'y16',
 'z16',
 'x17',
 'y17',
 'z17',
 'x18',
 'y18',
 'z18',
 'x19',
 'y19',
 'z19',
 'x20',
 'y20',
 'z20',
 'x21',
 'y21',
 'z21']

In [23]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [28]:
class_name = "Three"

In [29]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



        image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        try:
            hand = hand_landmarks.landmark
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())
            
            row = hand_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
            
   
        cv2.imshow('MediaPipe Hands 4', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

## Step 5: Read in Collected Data and Process

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [31]:
df = pd.read_csv('coords.csv')

X = df.drop('class', axis=1) # features
y = df['class'] # target value

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

## Step 6: Train Custom Model Using Scikit Learn

In [32]:

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [33]:

pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

## Step 7: Train Machine Learning Classification Model

In [34]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

c:\users\91992\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [35]:
fit_models['rc'].predict(X_test)

array(['One', 'Two', 'Three', 'Three', 'Three', 'Three', 'Two', 'Three',
       'One', 'Two', 'One', 'Three', 'Two', 'One', 'Two', 'Three',
       'Three', 'Three', 'One', 'One', 'Three', 'Three', 'One', 'Three',
       'Three', 'Three', 'Two', 'Three', 'Two', 'Two', 'Two', 'One',
       'Two', 'One', 'One', 'Two', 'Three', 'Three', 'Three', 'One',
       'One', 'Two', 'One', 'Two', 'One', 'Three', 'Two', 'Two', 'One',
       'One', 'One', 'Three', 'Two', 'Three', 'Two', 'One', 'Three',
       'Three', 'Three', 'Two', 'Two', 'Two', 'Two', 'Two', 'Three',
       'Three', 'Three', 'Two', 'Three', 'Two', 'Two', 'One', 'One',
       'Two', 'Three', 'Three', 'Three', 'Three', 'Two', 'Three', 'One',
       'One', 'One', 'Two', 'Three', 'Three', 'Three', 'Three', 'Three',
       'One', 'Two', 'Three', 'Three', 'One', 'Two', 'Three', 'Three',
       'Two', 'Three', 'One', 'One', 'Two', 'One', 'One', 'Two', 'Three',
       'Three', 'One', 'One', 'Three', 'One', 'One', 'One', 'Three',
       'On

## Step 8: Evaluate and Serialize Model

In [36]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [37]:

for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.94
rc 0.9266666666666666
rf 0.9466666666666667
gb 0.9466666666666667


In [38]:
fit_models['rf'].predict(X_test)

array(['One', 'Two', 'Three', 'Three', 'Three', 'Three', 'One', 'Three',
       'One', 'Two', 'One', 'Three', 'Two', 'One', 'Two', 'Three', 'One',
       'Three', 'One', 'One', 'Three', 'One', 'One', 'Three', 'Three',
       'Two', 'Two', 'Three', 'Two', 'Two', 'Three', 'One', 'Two', 'One',
       'One', 'Two', 'Three', 'Three', 'Three', 'One', 'One', 'Two',
       'One', 'Two', 'One', 'Three', 'Three', 'Two', 'One', 'One', 'One',
       'Three', 'Two', 'Two', 'Two', 'One', 'Three', 'Three', 'Three',
       'Two', 'Two', 'Two', 'Two', 'Two', 'Three', 'Three', 'Three',
       'Two', 'Three', 'Two', 'Two', 'One', 'One', 'Two', 'Three',
       'Three', 'Three', 'Three', 'Two', 'One', 'One', 'One', 'One',
       'Two', 'Three', 'Three', 'Three', 'Three', 'Three', 'One', 'Two',
       'Three', 'Three', 'One', 'Two', 'Three', 'One', 'Two', 'Three',
       'One', 'One', 'Two', 'One', 'One', 'Two', 'Three', 'Three', 'One',
       'One', 'Three', 'One', 'One', 'One', 'Three', 'One', 'Three',
  

In [39]:

with open('Numbers.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## Step 9: Make Detections with Model

In [40]:
with open('Numbers.pkl', 'rb') as f:
    model = pickle.load(f)

In [42]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
       
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



#         image_height, image_width, _ = image.shape

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        try:
            hand = hand_landmarks.landmark
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())

            row = hand_row
#             print(row)


            X = pd.DataFrame([row])
            hand_language_class = model.predict(X)[0]
            hand_language_prob = model.predict_proba(X)[0]
            print(hand_language_class, hand_language_prob)



            coords = tuple(np.multiply(
                            np.array(
                                (hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x, 
                                 hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y))
                        , [640,480]).astype(int))

            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(hand_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, hand_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, hand_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(hand_language_prob[np.argmax(hand_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass
            
   
        cv2.imshow('MediaPipe Hands 2', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Two [0.08 0.11 0.81]
Three [0.   0.83 0.17]
Three [0.   0.93 0.07]
Three [0.01 0.92 0.07]
Three [0.   0.99 0.01]
Three [0.   0.98 0.02]
Three [0.   0.97 0.03]
Three [0. 1. 0.]
Three [0. 1. 0.]
Three [0. 1. 0.]
Three [0.   0.94 0.06]
Two [0.08115385 0.15       0.76884615]
Two [0.17698718 0.05       0.77301282]
Two [0.11379779 0.04       0.84620221]
Two [0.10532051 0.04       0.85467949]
Two [0.0955348 0.05      0.8544652]
Two [0.0955348 0.04      0.8644652]
Two [0.11732051 0.04       0.84267949]
Two [0.11732051 0.04       0.84267949]
Two [0.1255348 0.05      0.8244652]
Two [0.09732051 0.05       0.85267949]
One [0.51464821 0.01       0.47535179]
Two [0.40417997 0.01       0.58582003]
Two [0.39292354 0.01       0.59707646]
Two [0.39318743 0.01       0.59681257]

Three [0.05 0.56 0.39]
Two [0.0535348 0.41      0.5364652]
Three [0.02 0.87 0.11]
Three [0.02 0.91 0.07]
Three [0.02 0.75 0.23]
Three [0.05 0.7  0.25]
Two [0.0435348 0.37      0.5864652]
Three [0.06115385 0.52       0.41884615]
Two [0.03115385 0.38       0.58884615]
Three [0.05115385 0.53       0.41884615]
Two [0.0535348 0.36      0.5864652]
Two [0.04115385 0.45       0.50884615]
Two [0.0535348 0.39      0.5564652]
Two [0.04115385 0.39       0.56884615]
Two [0.05115385 0.41       0.53884615]
Three [0.07346154 0.65       0.27653846]
Two [0.11821211 0.04       0.84178789]
Three [0.03115385 0.67       0.29884615]
Three [0.04 0.7  0.26]
Three [0.02 0.69 0.29]
Three [0.03 0.66 0.31]
Three [0.01 0.77 0.22]
Three [0.04238095 0.58       0.37761905]
Three [0.04 0.67 0.29]
Three [0.06 0.62 0.32]
Three [0.02 0.67 0.31]
Three [0.02 0.56 0.42]
Three [0.05238095 0.55       0.39761905]
Three [0.05 0.6  0.35]
Three [0.03 0.54 0.43]
Three [0.03 0.54 0.43]
Three [0.03 0.54 0.43]
Three [0.03 0.54 0.43]
T